###tinygrad

In [1]:
!git clone https://github.com/geohot/tinygrad.git

Cloning into 'tinygrad'...
remote: Enumerating objects: 13827, done.
remote: Counting objects: 100% (2596/2596), done.
remote: Compressing objects: 100% (349/349), done.
remote: Total 13827 (delta 2368), reused 2285 (delta 2247), pack-reused 11231
Receiving objects: 100% (13827/13827), 18.15 MiB | 34.80 MiB/s, done.
Resolving deltas: 100% (9780/9780), done.


In [2]:
cd tinygrad

/content/tinygrad


In [9]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/tinygrad
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.1/919.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pytools: filename=pytools-2022.1.14-py2.py3-none-any.whl size=69855 sha256=5292ec6a0479c842e54ad0325d64a8351269f55806911f6d87a6cb0a4980183b
  Stored in directory: /root/.cache/pip/wheels/19/02/16/aa2498ad7aa723a149ff7539f1918509661c0ae9d975b44b6d
Successfully built pytools
  Running setup.py develop for tinygrad


In [ ]:
from tinygrad.tensor import Tensor

x = Tensor.eye(3, requires_grad=True)
y = Tensor([[2.0,0,-2.0]], requires_grad=True)
z = y.matmul(x).sum()
z.backward()

print(x.grad.numpy())  # dz/dx
print(y.grad.numpy())  # dz/dy

[[ 2.  2.  2.]
 [ 0.  0.  0.]
 [-2. -2. -2.]]
[[1. 1. 1.]]


In [ ]:
from tinygrad.tensor import Tensor;
N = 1024; a, b = Tensor.randn(N, N), Tensor.randn(N, N);
c = (a.reshape(N, 1, N) * b.permute(1,0).reshape(1, N, N)).sum(axis=2);
print((c.numpy() - (a.numpy() @ b.numpy())).mean())

1.7190548e-08


In [ ]:
%%writefile fast.sh

DEBUG=3 OPTLOCAL=1 python3 -c "from tinygrad.tensor import Tensor;
N = 1024; a, b = Tensor.randn(N, N), Tensor.randn(N, N);
c = (a.reshape(N, 1, N) * b.permute(1,0).reshape(1, N, N)).sum(axis=2);
print((c.numpy() - (a.numpy() @ b.numpy())).mean())"

Writing fast.sh


In [ ]:
!sh fast.sh

*** exec  0.01 GB    0.10 ms op: MovementOps.RESHAPE  out(float): (1024, 1, 1024)                in(1): [(1024, 1024)] (1024, 1, 1024)
*** exec  4.30 GB    0.17 ms op: MovementOps.EXPAND   out(float): (1024, 1024, 1024)             in(1): [(1024, 1, 1024)] (1024, 1024, 1024)
*** exec  4.31 GB    0.10 ms op: MovementOps.PERMUTE  out(float): (1024, 1024)                   in(1): [(1024, 1024)] (1, 0)
*** exec  4.31 GB    0.10 ms op: MovementOps.RESHAPE  out(float): (1, 1024, 1024)                in(1): [(1024, 1024)] (1, 1024, 1024)
*** exec  8.60 GB    0.13 ms op: MovementOps.EXPAND   out(float): (1024, 1024, 1024)             in(1): [(1, 1024, 1024)] (1024, 1024, 1024)
*** exec  8.60 GB  420.41 ms op: FusedOps.MULACC      out(float): (1024, 1024, 1)                in(2): [(1024, 1024, 1024)] (1024, 1024, 1)
*** exec  0.02 GB    0.11 ms op: MovementOps.RESHAPE  out(float): (1024, 1024)                   in(1): [(1024, 1024, 1)] (1024, 1024)
3.792106e-09


In [ ]:
!python3 -m pip install -e '.[testing]'

In [ ]:
!python -m pytest

In [ ]:
!DEBUG=3 OPTLOCAL=1 python3 -c "from tinygrad.tensor import Tensor; \
N = 1024; a, b = Tensor.randn(N, N), Tensor.randn(N, N); \
c = (a.reshape(N, 1, N) * b.permute(1,0).reshape(1, N, N)).sum(axis=2); \
print((c.numpy() - (a.numpy() @ b.numpy())).mean())"

*** exec  0.00 GB    0.10 ms op: MovementOps.RESHAPE  out(float): (1, 1)                         in(1): [()] (1, 1)
*** exec  0.00 GB    0.18 ms op: MovementOps.EXPAND   out(float): (1024, 1024)                   in(1): [(1, 1)] (1024, 1024)
*** exec  0.02 GB    0.12 ms op: MovementOps.RESHAPE  out(float): (2, 1024, 1024)                in(1): [(2097152,)] (2, 1024, 1024)
*** exec  0.02 GB    0.10 ms op: MovementOps.SHRINK   out(float): (1, 1024, 1024)                in(1): [(2, 1024, 1024)] ((0, 1), (0, 1024), (0, 1024))
*** exec  0.02 GB    0.09 ms op: MovementOps.RESHAPE  out(float): (1024, 1024)                   in(1): [(1, 1024, 1024)] (1024, 1024)
*** exec  0.02 GB    0.07 ms op: MovementOps.RESHAPE  out(float): (1, 1)                         in(1): [()] (1, 1)
*** exec  0.02 GB    0.11 ms op: MovementOps.EXPAND   out(float): (1024, 1024)                   in(1): [(1, 1)] (1024, 1024)
*** exec  0.03 GB    0.12 ms op: MovementOps.SHRINK   out(float): (1, 1024, 1024)              

In [8]:
!python -m pip install -U mypy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 99.5 MB/s eta 0:00:00


In [13]:
!mypyc --check-untyped-defs --explicit-package-bases --warn-unreachable tinygrad/shape/shapetracker.py tinygrad/shape/symbolic.py \
  tinygrad/helpers.py tinygrad/mlops.py tinygrad/tensor.py tinygrad/graph.py

Traceback (most recent call last):
  File "/content/tinygrad/build/setup.py", line 1, in <module>
    from setuptools import setup
  File "/usr/local/lib/python3.10/dist-packages/setuptools/__init__.py", line 7, in <module>
    import _distutils_hack.override  # noqa: F401
  File "/usr/local/lib/python3.10/dist-packages/_distutils_hack/override.py", line 1, in <module>
    __import__('_distutils_hack').do_override()
  File "/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py", line 77, in do_override
    ensure_local_distutils()
  File "/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py", line 63, in ensure_local_distutils
    core = importlib.import_module('distutils.core')
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/core.py", line 22, in <module>
    from .dist import Distribution
  Fi